In [1]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
from collections import Counter
import evalml
from evalml import AutoMLSearch
from evalml.utils import infer_feature_types
import torch
from torch.autograd import Variable
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix,accuracy_score, roc_curve, auc, precision_recall_curve, average_precision_score
from sklearn.metrics import roc_auc_score, confusion_matrix,accuracy_score, roc_curve, auc, precision_recall_curve
import time

The next non-bugfix release of Featuretools will not support Python 3.6


https://www.va.gov/MS/Professionals/diagnosis/Kurtzke_Expanded_Disability_Status_Scale.asp

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 1000)

In [3]:
folder = '../ai_training_set'
filelists = os.listdir(folder)

for file in filelists:
    if file not in ['Updated Database.xlsx', '.ipynb_checkpoints', 'AI_Notes']:
        print("df_" + file[:-4] + "=pd.read_csv('" + folder +"/" + file + "')")
        exec("df_" + file[:-4] + "=pd.read_csv('" + folder +"/" + file + "')")
        print("display(" + "df_" + file[:-4] + ".head(10))")
        exec("display(" + "df_" + file[:-4] + ".head(10))")

df_AI_patient_facts=pd.read_csv('../ai_training_set/AI_patient_facts.csv')
display(df_AI_patient_facts.head(10))


,uid,PatientID,MRN,LastEditedDTTM,Race,RaceCode,OMBRaceCode,Ethnicity,OMBEthnicCode,MaritalStatus,PrimaryLanguage,PrimaryInsurance,IsBTG
0,2045655,444345,67552,2021-01-05 01:12:45.443,Caucasian,Caucasian,2,Non-Hispanic or Latino,3,Unknown,ENGLISH,MERITAIN HEALTH/PO BOX 27267/MINNEAPOLIS,N
1,1968324,5841,107204,2020-06-12 21:59:12.967,Caucasian,Caucasian,2,Non-Hispanic or Latino,3,Unknown,ENGLISH,AMERIGROUP/PO BOX 61010/VIRGINIA BEACH,N
2,2145820,6682,108532,2021-05-10 09:06:17.010,Unknown,U,0,Declined,4,Unknown,Unknown,BLUE CROSS HEALTHSELE PPO,N
3,1473756,10117,114410,2018-06-28 17:43:07.400,African American,B,4,Non-Hispanic or Latino,3,Married,ENGLISH,BCBS OF TEXAS/PO BOX 660044/DALLAS,N
4,2179716,484328,1146923,2021-06-01 15:24:27.243,Unknown,U,0,Declined,4,Unknown,Unknown,COMMUNITY HEALTH EXCHANGE,N
5,1883256,490822,1159182,2021-05-27 00:02:22.243,Unknown,U,0,Declined,4,Unknown,Unknown,AETNA MEM HERMANN EMP,N
6,1080175,524281,1219155,2021-03-08 01:32:38.993,Caucasian,Caucasian,2,Non-Hispanic or Latino,3,Married,ENGLISH,AETNA/PO BOX 981106/EL PASO,N
7,1335335,533949,1237033,2020-11-16 09:38:35.350,African American,B,4,Non-Hispanic or Latino,3,Unknown,ENGLISH,CIGNA/PO BOX 182223/CHATTANOOGA,N
8,603352,536669,1241985,2021-05-05 10:15:34.517,Latin American,S,2,Hispanic or Latino,2,Married,ENGLISH,COMMUNITY HEALTH CHOICE SILVER/PO BOX 301424/H...,N
9,2180222,537738,1243867,2021-04-09 02:24:07.837,Hispanic or Latino,L,2,Hispanic or Latino,2,Single,ENGLISH,UHC COMMUNITY STAR/CHIP/MEDICARE SNP/PO BOX 52...,N


df_AI_diagnoses=pd.read_csv('../ai_training_set/AI_diagnoses.csv')
display(df_AI_diagnoses.head(10))


,uid_src,uid,PersonID,MRN,INV_NUM,VIS_NUM,Provider,ProviderID,Location,Department,Division,LocationType,SERVICE_DATE,ADMIT_DATE,DX,DX_NUM,CDW_UpdatedDTTM,CDW_RunID,CDW_i2b2_UploadID
0,mpi,1744190,2429008,3334154,14087248,3484373.0,"LINDSEY MD,JOHN W",25181,UTP HPB CLINIC,NEU-MULTIPLE SCLEROSIS-4,NEUROLOGY,OUTPATIENT,2009-12-02 00:00:00.000,NaN,341.9,1,2021-06-12 22:12:37.400,0,0
1,mpi,1744190,2429008,3334154,14087248,3484373.0,"LINDSEY MD,JOHN W",25181,UTP HPB CLINIC,NEU-MULTIPLE SCLEROSIS-4,NEUROLOGY,OUTPATIENT,2009-12-02 00:00:00.000,NaN,340,2,2021-06-12 22:12:37.400,0,0
2,mpi,1744190,2429008,3334154,16957811,4270516.0,"NELSON MD,FLAVIA",25219,UTP HPB CLINIC,NEU-MULTIPLE SCLEROSIS-4,NEUROLOGY,OUTPATIENT,2011-07-06 00:00:00.000,NaN,340,1,2021-06-12 22:12:37.400,0,0
3,mpi,1744190,2429008,3334154,16957811,4270516.0,"NELSON MD,FLAVIA",25219,UTP HPB CLINIC,NEU-MULTIPLE SCLEROSIS-4,NEUROLOGY,OUTPATIENT,2011-07-06 00:00:00.000,NaN,782.0,2,2021-06-12 22:12:37.400,0,0
4,mpi,1744190,2429008,3334154,16957811,4270516.0,"NELSON MD,FLAVIA",25219,UTP HPB CLINIC,NEU-MULTIPLE SCLEROSIS-4,NEUROLOGY,OUTPATIENT,2011-07-06 00:00:00.000,NaN,293.84,3,2021-06-12 22:12:37.400,0,0
5,mpi,1744190,2429008,3334154,19122635,4899519.0,"NELSON MD,FLAVIA",25219,UTP HPB CLINIC,NEU-MULTIPLE SCLEROSIS-4,NEUROLOGY,OUTPATIENT,2012-08-08 00:00:00.000,NaN,340,1,2021-06-12 22:12:37.400,0,0
6,mpi,1744190,2429008,3334154,19122635,4899519.0,"NELSON MD,FLAVIA",25219,UTP HPB CLINIC,NEU-MULTIPLE SCLEROSIS-4,NEUROLOGY,OUTPATIENT,2012-08-08 00:00:00.000,NaN,311,2,2021-06-12 22:12:37.400,0,0
7,mpi,1744190,2429008,3334154,19122635,4899519.0,"NELSON MD,FLAVIA",25219,UTP HPB CLINIC,NEU-MULTIPLE SCLEROSIS-4,NEUROLOGY,OUTPATIENT,2012-08-08 00:00:00.000,NaN,782.0,3,2021-06-12 22:12:37.400,0,0
8,mpi,1744190,2429008,3334154,19122635,4899519.0,"NELSON MD,FLAVIA",25219,UTP HPB CLINIC,NEU-MULTIPLE SCLEROSIS-4,NEUROLOGY,OUTPATIENT,2012-08-08 00:00:00.000,NaN,293.84,4,2021-06-12 22:12:37.400,0,0
9,mpi,1744190,2429008,3334154,20262873,5281810.0,"NELSON MD,FLAVIA",25219,UTP HPB CLINIC,NEU-MULTIPLE SCLEROSIS-4,NEUROLOGY,OUTPATIENT,2013-02-08 00:00:00.000,NaN,340,1,2021-06-12 22:12:37.400,0,0


df_AI_notes=pd.read_csv('../ai_training_set/AI_notes.csv')
display(df_AI_notes.head(10))


...

,Unnamed: 0,Unnamed: 0.1,doc_id,id,resourceType,coding,text,ingredient,form,rxnorm,DocumentID,E_ID,MRN
0,0,0,11538944.txt,b761b6a9-3371-39cc-8df3-6027e6aebaef,Medication,"[{'code': '5489', 'system': 'http://www.nlm.ni...",Hydrocodone,[{'itemCodeableConcept': {'text': 'Hydrocodone...,NaN,5489,11538944,68742495,3334154.0
1,1,1,12771238.txt,6d8db5ef-523f-35a6-926b-f6940d394c61,Medication,"[{'code': '141927', 'system': 'http://www.nlm....",Xanax,"[{'amount': {'numerator': {'unit': 'MG', 'valu...",{'text': 'Oral Tablet'},141927,12771238,9461525,3334154.0
2,2,2,13685231.txt,452e45e5-fddf-3fe7-b581-4aa753a8f509,Medication,"[{'code': '197582', 'system': 'http://www.nlm....",Dexamethasone,"[{'amount': {'numerator': {'unit': 'MG', 'valu...",{'text': 'Oral Tablet'},197582,13685231,9818181,3334154.0
3,3,3,14375248.txt,0f1480c7-f3ec-36c7-953c-cb39cf099c52,Medication,"[{'code': '993517', 'system': 'http://www.nlm....",Bupropion Hydrochloride,"[{'amount': {'numerator': {'unit': 'MG', 'valu...",{'text': 'Oral Tablet'},993517,14375248,9963456,4158906.0
4,4,4,17124192.txt,2470dd38-79ee-33cd-a9bf-06175f6773e4,Medication,"[{'code': '315565', 'system': 'http://www.nlm....",Carbamazepine,"[{'amount': {'numerator': {'unit': 'MG', 'valu...",{'text': 'Oral Capsule'},315565,17124192,10632816,3334154.0
5,5,5,8413195.txt,48d2c4b1-d1de-35b0-88d0-d2ce00bbd744,Medication,"[{'code': '135779', 'system': 'http://www.nlm....",Copaxone,"[{'amount': {'numerator': {'unit': 'MG/ML', 'v...",NaN,135779,8413195,11615838,4158906.0
6,6,6,18504284.txt,d38b53f7-19c2-36b2-b538-d03b4e20cabe,Medication,"[{'code': '89905', 'system': 'http://www.nlm.n...",Multivitamin preparation,"[{'amount': {'denominator': {'value': 1}}, 'it...",{'text': 'Oral Tablet'},89905,18504284,11615838,4158906.0
7,7,7,18722559.txt,727cb8ea-3981-393c-ab01-fbf2638995e2,Medication,"[{'code': '284245', 'system': 'http://www.nlm....",Famotidine,"[{'amount': {'numerator': {'unit': 'MG', 'valu...",{'text': 'Oral Tablet'},284245,18722559,12401350,3334154.0
8,8,8,18722559.txt,fe5ce2f4-4e1e-320e-a841-17ff881eb382,Medication,"[{'code': '834022', 'system': 'http://www.nlm....",Ketorolac Tromethamine,"[{'amount': {'numerator': {'unit': 'MG', 'valu...",{'text': 'Oral Tablet'},834022,18722559,12498587,3377497.0
9,9,9,19013140.txt,c51fabec-2ef9-386e-837d-324290b188b0,Medication,"[{'code': '151672', 'system': 'http://www.nlm....",Dulcolax,"[{'amount': {'numerator': {'unit': 'MG', 'valu...",{'text': 'Oral Tablet'},151672,19013140,12556725,3377497.0


df_AI_Notes_medicationstatement_MRN=pd.read_csv('../ai_training_set/AI_Notes_medicationstatement_MRN.csv')
display(df_AI_Notes_medicationstatement_MRN.head(10))


,Unnamed: 0,Unnamed: 0.1,doc_id,dosage,id,resourceType,reasonForUseCodeableConcept,0,coding,text,snomed,Reason,DocumentID,E_ID,MRN
0,0,0,12369461.txt,"[{'doseQuantity': {'value': 1}, 'asNeeded': {'...",16de57d3-a2f0-3208-84e9-d11b997b796b,MedicationStatement,NaN,{'value': 'urn:uuid:16de57d3-a2f0-3208-84e9-d1...,"[{'code': '197901', 'system': 'http://www.nlm....",Lorazepam,197901,NaN,12369461,68742495,3334154.0
1,1,1,12369461.txt,"[{'timing': {'repeat': {'periodUnit': 'd', 'pe...",ad84b2a4-4291-3ef5-8e10-8ae9683ace3c,MedicationStatement,NaN,{'value': 'urn:uuid:ad84b2a4-4291-3ef5-8e10-8a...,"[{'code': '135779', 'system': 'http://www.nlm....",Copaxone,135779,NaN,12369461,9461525,3334154.0
2,2,2,12536158.txt,"[{'doseQuantity': {'unit': 'Oral Tablet', 'val...",1719afd6-8753-35d2-9f9f-71ad29d7ff36,MedicationStatement,NaN,{'value': 'urn:uuid:1719afd6-8753-35d2-9f9f-71...,"[{'code': '197582', 'system': 'http://www.nlm....",Dexamethasone,197582,NaN,12536158,9818181,3334154.0
3,3,3,15177132.txt,NaN,01cc9785-68f4-3f98-bcf4-0c11f16741fb,MedicationStatement,"[{'text': 'Disease', 'coding': [{'code': '6457...",{'value': 'urn:uuid:01cc9785-68f4-3f98-bcf4-0c...,"[{'code': '5666', 'system': 'http://www.nlm.ni...",Immunoglobulin G,5666,"['Disease', 'Disease', 'Lyme Disease', 'Lyme D...",15177132,9963456,4158906.0
4,4,4,17124192.txt,"[{'doseQuantity': {'unit': 'Oral Capsule', 'va...",bf937ede-75cd-3b0a-a38e-df3df59577eb,MedicationStatement,NaN,{'value': 'urn:uuid:bf937ede-75cd-3b0a-a38e-df...,"[{'code': '315565', 'system': 'http://www.nlm....",Carbamazepine,315565,NaN,17124192,10632816,3334154.0
5,5,5,8413195.txt,"[{'timing': {'repeat': {'periodUnit': 'd', 'pe...",ef89dbe0-2c97-38a8-b857-cfd6adc76aa9,MedicationStatement,NaN,{'value': 'urn:uuid:ef89dbe0-2c97-38a8-b857-cf...,"[{'code': '135779', 'system': 'http://www.nlm....",Copaxone,135779,NaN,8413195,11615838,4158906.0
6,6,6,84141606.txt,NaN,6bed9199-aa02-3937-8e67-f798aee90990,MedicationStatement,NaN,{'value': 'urn:uuid:6bed9199-aa02-3937-8e67-f7...,"[{'code': '174742', 'system': 'http://www.nlm....",Plavix,174742,NaN,84141606,11615838,4158906.0
7,7,7,18245789.txt,"[{'doseQuantity': {'unit': 'Oral Tablet', 'val...",ac149d9a-589e-3eb1-b41d-033dfd1de6bf,MedicationStatement,NaN,{'value': 'urn:uuid:ac149d9a-589e-3eb1-b41d-03...,"[{'code': '89905', 'system': 'http://www.nlm.n...",Multivitamin preparation,89905,NaN,18245789,12401350,3334154.0
8,8,8,19013140.txt,NaN,c9f202e1-6e47-3a28-af77-28a124117133,MedicationStatement,NaN,{'value': 'urn:uuid:c9f202e1-6e47-3a28-af77-28...,"[{'code': '11253', 'system': 'http://www.nlm.n...",Vitamin D,11253,NaN,19013140,12498587,3377497.0
9,9,9,2119704.txt,"[{'timing': {'repeat': {'duration': 2, 'durati...",da1f7512-397d-3dcb-ae2d-5e657146c298,MedicationStatement,NaN,{'value': 'urn:uuid:da1f7512-397d-3dcb-ae2d-5e...,"[{'code': '135779', 'system': 'http://www.nlm....",Copaxone,135779,NaN,2119704,12556725,3377497.0


df_AI_Notes_procedures_MRN=pd.read_csv('../ai_training_set/AI_Notes_procedures_MRN.csv')
display(df_AI_Notes_procedures_MRN.head(10))


,Unnamed: 0,Unnamed: 0.1,doc_id,id,resourceType,coding,text,0,snomed,DocumentID,E_ID,MRN
0,0,0,1176110.txt,6f184832-310f-3d43-b76e-908daa2e2d2a,Procedure,"[{'code': '39857003', 'system': 'http://snomed...",weight,{'value': 'urn:uuid:6f184832-310f-3d43-b76e-90...,39857003.0,1176110,68742495,3334154.0
1,1,1,1176110.txt,7085525e-5bd7-318c-a29e-d239095aac48,Procedure,"[{'code': '410560002', 'system': 'http://snome...",confrontation,{'value': 'urn:uuid:7085525e-5bd7-318c-a29e-d2...,410560002.0,1176110,9461525,3334154.0
2,2,2,12771238.txt,b70f30f6-712d-3c91-a810-924c3af37768,Procedure,"[{'code': '115956009', 'system': 'http://snome...",Release,{'value': 'urn:uuid:b70f30f6-712d-3c91-a810-92...,115956009.0,12771238,9818181,3334154.0
3,3,3,14319491.txt,29db130a-2f0e-327c-8745-27e40fa25f78,Procedure,NaN,ASSAY,{'value': 'urn:uuid:29db130a-2f0e-327c-8745-27...,NaN,14319491,9963456,4158906.0
4,4,4,15177132.txt,2fe64016-f43c-3587-be5a-4ad6014410a0,Procedure,"[{'code': '113091000', 'system': 'http://snome...",MRI,{'value': 'urn:uuid:2fe64016-f43c-3587-be5a-4a...,113091000.0,15177132,10632816,3334154.0
5,5,5,16395149.txt,ebe78baa-9d93-300c-8247-bc64687114b6,Procedure,"[{'code': '115956009', 'system': 'http://snome...",Release,{'value': 'urn:uuid:ebe78baa-9d93-300c-8247-bc...,115956009.0,16395149,11615838,4158906.0
6,6,6,16395149.txt,8fe8d25d-14ae-39a5-90bc-e2d71831c935,Procedure,"[{'code': '29567006', 'system': 'http://snomed...",MRI Brain,{'value': 'urn:uuid:8fe8d25d-14ae-39a5-90bc-e2...,29567006.0,16395149,11615838,4158906.0
7,7,7,17124192.txt,730226a6-0894-351e-9ba1-eac5132150ab,Procedure,"[{'code': '113091000', 'system': 'http://snome...",MRI,{'value': 'urn:uuid:730226a6-0894-351e-9ba1-ea...,113091000.0,17124192,12401350,3334154.0
8,8,8,83356759.txt,beee2340-883d-3812-9326-76aa19bb53fd,Procedure,NaN,Analysis,{'value': 'urn:uuid:beee2340-883d-3812-9326-76...,NaN,83356759,12498587,3377497.0
9,9,9,83843309.txt,7d51ae0a-ae49-33b8-bdd9-c4d78866e6fa,Procedure,"[{'code': '26604007', 'system': 'http://snomed...",CBC,{'value': 'urn:uuid:7d51ae0a-ae49-33b8-bdd9-c4...,26604007.0,83843309,12556725,3377497.0


In [4]:
df_AI_labs[df_AI_labs['MRN'] == 4158906].sort_values(by='IR_PerformedDTTM')['IR_PerformedDTTM']

61    2013-02-27 13:45:00.000
14    2013-02-27 13:45:00.000
31    2013-02-27 13:45:00.000
32    2013-02-27 13:45:00.000
45    2013-02-27 13:45:00.000
46    2013-02-27 13:45:00.000
47    2013-02-27 13:45:00.000
60    2013-02-27 13:45:00.000
8     2013-02-27 13:45:00.000
6     2013-02-27 13:45:00.000
5     2013-02-27 13:45:00.000
4     2013-02-27 13:45:00.000
48    2013-02-27 13:45:00.000
49    2013-02-27 13:45:00.000
7     2013-02-27 13:45:00.000
16    2013-02-27 13:45:00.000
3     2013-02-27 15:43:00.000
0     2013-02-27 15:44:00.000
11    2013-08-27 08:51:00.000
10    2013-08-27 08:51:00.000
9     2013-08-27 13:17:00.000
26    2014-09-26 10:18:00.000
52    2014-09-26 10:18:00.000
44    2014-09-26 10:18:00.000
43    2014-09-30 06:00:00.000
42    2014-09-30 06:00:00.000
41    2014-09-30 06:00:00.000
53    2014-09-30 06:00:00.000
38    2014-09-30 06:00:00.000
37    2014-09-30 06:00:00.000
36    2014-09-30 06:00:00.000
35    2014-09-30 06:00:00.000
55    2014-09-30 06:00:00.000
54    2014

In [5]:
for file in filelists:
    print("df_" + file[:-4])

df_AI_N
df_.ipynb_checkpo
df_AI_patient_facts
df_AI_diagnoses
df_AI_notes
df_AI_labs
df_AI_encounters
df_AI_vitals
df_AI_medications
df_AI_procedures
df_AI_demographics
df_Updated Database.
df_AI_medications_MRN
df_AI_notes_MRN
df_AI_Notes_condition_MRN
df_AI_Notes_med_MRN
df_AI_Notes_medicationstatement_MRN
df_AI_Notes_procedures_MRN


In [6]:
for i in df_AI_notes.columns:
    print(i)

DocumentId
DocumentTypeDE
DocumentType
status
RecordedDTTM
LastEditDTTM
EncounterId
EditableChunk
UneditableChunk
PlainText


In [7]:
print('df_AI_demographics')
display(df_AI_demographics.head(3))
print('df_AI_patient_facts')
display(df_AI_patient_facts.head(3))
print('df_AI_labs')
display(df_AI_labs.head(3))
print('df_AI_vitals')
display(df_AI_vitals.head(3))
print('df_AI_diagnoses')
display(df_AI_diagnoses.head(3))
print('df_AI_notes')
display(df_AI_notes.head(3))
print('df_AI_medications')
display(df_AI_medications.head(3))
print('df_AI_encounters')
display(df_AI_encounters.head(3))
print('df_AI_procedures')
display(df_AI_procedures.head(3))


df_AI_demographics


,PatientID,Uid,MRN,FirstName,LastName,DateOfBirth,Race,Sex,isDeceased,MaritalStatus,ZipCode,ZipSafe,AgeSafe,IgnoreRow
0,1076404,1260414,1938659,LAURA,SALDATE,1970-08-21,OTHER,F,N,MARRIED,77064,77064,51,0
1,610834,2484072,1378510,CHALITA,DAVIS BABINEAUX,1972-04-05,AFRICAN AMERICAN,F,N,UNKNOWN,77433,77433,49,0
2,4807039,5048900,5912851,LISA,BABINEAUX,1970-04-10,AFRICAN AMERICAN,F,N,SINGLE,77086,77086,51,0


df_AI_patient_facts


,uid,PatientID,MRN,LastEditedDTTM,Race,RaceCode,OMBRaceCode,Ethnicity,OMBEthnicCode,MaritalStatus,PrimaryLanguage,PrimaryInsurance,IsBTG
0,2045655,444345,67552,2021-01-05 01:12:45.443,Caucasian,Caucasian,2,Non-Hispanic or Latino,3,Unknown,ENGLISH,MERITAIN HEALTH/PO BOX 27267/MINNEAPOLIS,N
1,1968324,5841,107204,2020-06-12 21:59:12.967,Caucasian,Caucasian,2,Non-Hispanic or Latino,3,Unknown,ENGLISH,AMERIGROUP/PO BOX 61010/VIRGINIA BEACH,N
2,2145820,6682,108532,2021-05-10 09:06:17.010,Unknown,U,0,Declined,4,Unknown,Unknown,BLUE CROSS HEALTHSELE PPO,N


df_AI_labs


...

,VisitID,EncounterID,PatientID,M_ID,M_ItemID,M_ndc,M_MedDict,M_Dayssupply,M_MedDictNDC,M_MedDictDDI,M_MedDictStrength,M_MedDictUnits,M_FreeTextPrescriber,M_FreeTextSIGFLAG,IM_PerformedDTTM,IM_EntryName
0,8317474,9963456,3221151,10193584700019,3623025900011,55289034630,Xanax 0.25 MG Oral Tablet,0,00009002902,24002,0.25,MG,NaN,Y,2015-12-09 00:00:00.000,Xanax 0.25 MG Oral Tablet
1,8317474,9963456,3221151,4421835500005,3623025900007,68071004060,Zoloft TABS,0,00049496030,49174,NaN,,NaN,N,2013-08-28 13:57:04.470,Zoloft TABS
2,8317474,9963456,3221151,4421835500001,3629126700009,55390021901,MethylPREDNISolone Sodium Succ 1 GM SOLR,5,55390021901,125362,1,GM,NaN,Y,2013-02-28 00:00:00.000,MethylPREDNISolone Sodium Succ 1 GM Injection ...


df_AI_encounters


,E_ID,PatientID,MRN,UID,E_VisitID,E_DTTM,LastUpdatedDTTM,E_EncounterTypeDE,E_EncounterType,E_ProviderID,E_Provider1,PR1_SpecialtyDE,PR1_Specialty,PR1_Specialty2DE,PR1_Specialty2,E_Provider2ID,E_Provider2,PR2_SpecialtyDE,PR2_Specialty,PR2_Specialty2DE,PR2_Specialty2,E_ReferringProviderID,E_ReferringProvider
0,43753463,3331691,4276751,3278281,31010304,2018-07-13 09:41:02.910,2018-07-13 11:45:21.750,24,Chart Update,3959054,"Creag, Erliza",0,,0.0,,3959054,"Creag, Erliza",0,,0.0,,0,NaN
1,42374056,3331691,4276751,3278281,30061785,2018-05-21 11:41:00.000,2018-05-22 19:15:56.090,16,Lab Encounter,4039736,"FREEMAN, LEORAH",51,Neurology,0.0,,0,NaN,51,Neurology,0.0,,0,NaN
2,45846252,3331691,4276751,3278281,32487681,2018-09-21 11:39:32.303,2018-09-21 17:45:25.920,24,Chart Update,6033779,"SAMUDRALWAR, ROHINI",51,Neurology,0.0,,6033779,"SAMUDRALWAR, ROHINI",51,Neurology,0.0,,0,NaN


df_AI_procedures


,uid_src,uid,PersonID,MRN,INV_NUM,VIS_NUM,Provider,ProviderID,Location,Department,Division,LocationType,SERVICE_DATE,ADMIT_DATE,PROC,TXN_NUM,PROC_NUM,CDW_UpdatedDTTM,CDW_RunID,CDW_i2b2_UploadID
0,mpi,1086726,2169889,3069102,15839685,NaN,"SUPSUPIN JR MD,EMILIO P",30159,IP MHH,RAD-VASCULAR INTERVENTIONAL-4,RADIOLOGY,INPATIENT,2010-11-22 00:00:00.000,2010-11-22 00:00:00.000,77003,1,2,2021-06-14 11:14:36.270,0,0
1,mpi,1086726,2169889,3069102,15839685,NaN,"SUPSUPIN JR MD,EMILIO P",30159,IP MHH,RAD-VASCULAR INTERVENTIONAL-4,RADIOLOGY,INPATIENT,2010-11-22 00:00:00.000,2010-11-22 00:00:00.000,77003,1,1,2021-06-14 11:14:36.270,0,0
2,mpi,1086726,2169889,3069102,15839686,NaN,"SCHIESS MD,MYA",25202,IP MHH,NEU-MOVEMENT DISORDER-4,NEUROLOGY,INPATIENT,2010-11-22 00:00:00.000,2010-11-22 00:00:00.000,99236,1,1,2021-06-14 11:14:36.270,0,0


In [18]:
df_AI_medications

,VisitID,EncounterID,PatientID,M_ID,M_ItemID,M_ndc,M_MedDict,M_Dayssupply,M_MedDictNDC,M_MedDictDDI,M_MedDictStrength,M_MedDictUnits,M_FreeTextPrescriber,M_FreeTextSIGFLAG,IM_PerformedDTTM,IM_EntryName
0,8317474,9963456,3221151,10193584700019,3623025900011,55289034630,Xanax 0.25 MG Oral Tablet,0,00009002902,24002,0.25,MG,NaN,Y,2015-12-09 00:00:00.000,Xanax 0.25 MG Oral Tablet
1,8317474,9963456,3221151,4421835500005,3623025900007,68071004060,Zoloft TABS,0,00049496030,49174,NaN,,NaN,N,2013-08-28 13:57:04.470,Zoloft TABS
2,8317474,9963456,3221151,4421835500001,3629126700009,55390021901,MethylPREDNISolone Sodium Succ 1 GM SOLR,5,55390021901,125362,1,GM,NaN,Y,2013-02-28 00:00:00.000,MethylPREDNISolone Sodium Succ 1 GM Injection ...
3,8317474,9963456,3221151,4421835500009,3622891300007,54868406700,TEGretol-XR 100 MG Oral Tablet Extended Releas...,30,00078051005,44441,100,MG,NaN,Y,2013-04-22 00:00:00.000,TEGretol XR 100 MG Oral Tablet Extended Releas...
4,8317474,9963456,3221151,3629126700011,3629126700009,,MethylPREDNISolone Sodium Succ 1 GM SOLR,5,55390021901,125362,1,GM,NaN,Y,2013-02-28 00:00:00.000,MethylPREDNISolone Sodium Succ 1 GM Injection ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27727,52927354,74577737,4841174,28892668800000,28890461000005,51862094210,diazePAM 5 MG Oral Tablet,1,51862094210,6507,5,MG,NaN,Y,2021-06-11 00:00:00.000,diazePAM 5 MG Oral Tablet
27728,52927354,74577737,4841174,28892669000000,28890461000005,51862094210,diazePAM 5 MG Oral Tablet,1,51862094210,6507,5,MG,NaN,Y,2021-06-11 00:00:00.000,diazePAM 5 MG Oral Tablet
27729,52927354,74577737,4841174,28892668900000,28890461000005,51862094210,diazePAM 5 MG Oral Tablet,1,51862094210,6507,5,MG,NaN,Y,2021-06-11 00:00:00.000,diazePAM 5 MG Oral Tablet
27730,52927354,74577737,4841174,28891632700001,28890461000005,51862094210,diazePAM 5 MG Oral Tablet,1,51862094210,6507,5,MG,NaN,Y,2021-06-11 00:00:00.000,diazePAM 5 MG Oral Tablet


In [25]:
# df_AI_medications[df_AI_medications['M_MedDict']=='Topiramate 100 MG Oral Tablet']
# df_AI_medications[df_AI_medications['M_MedDict']=='Topiramate 50 MG Oral Tablet']
df_AI_medications['M_ndc'].nunique(),\
df_AI_medications['M_ItemID'].nunique(),\
df_AI_medications['M_ID'].nunique()

(2445, 5235, 27732)

### Patient EDSS scores


In [8]:
df_edss_score_EDSS = pd.read_excel('../ai_training_set/Updated Database.xlsx', sheet_name='(5) EDSS')
df_edss_score_MRI = pd.read_excel('../ai_training_set/Updated Database.xlsx', sheet_name='(6) MRI')

df_edss_score_MRI = df_edss_score_MRI.dropna(subset=['record_id', 'allscripts_mrn'])

## record_id --- allscripts_mrn
record_id_TO_allscripts_mrn = {record_id: allscripts_mrn for record_id, allscripts_mrn in zip(df_edss_score_MRI['record_id'].to_list(), df_edss_score_MRI['allscripts_mrn'].to_list())}

df_edss_score_EDSS = df_edss_score_EDSS.replace({"record_id": record_id_TO_allscripts_mrn}).rename(columns={'record_id': 'MRN'})

df_edss_score_EDSS = df_edss_score_EDSS.dropna().reset_index(drop=True)

result_EDSS = dict()  #{patient: {date: value}}
for i in range(df_edss_score_EDSS.shape[0]):
    mrn = df_edss_score_EDSS.loc[i, 'MRN']
    date = df_edss_score_EDSS.loc[i, 'dov']
    edss = df_edss_score_EDSS.loc[i, 'visit_edss']
    if mrn not in result_EDSS:
        result_EDSS[mrn] = dict()
    if date not in result_EDSS[mrn]:
        result_EDSS[mrn][date] = 0
    result_EDSS[mrn][date] = edss
    
result_EDSS



{3790917.0: {Timestamp('2012-04-12 00:00:00'): 1.0,
  Timestamp('2014-05-08 00:00:00'): 1.0,
  Timestamp('2015-11-23 00:00:00'): 1.0,
  Timestamp('2020-01-06 00:00:00'): 1.0},
 4782217.0: {Timestamp('2016-05-31 00:00:00'): 1.5,
  Timestamp('2017-01-10 00:00:00'): 2.0,
  Timestamp('2018-07-10 00:00:00'): 2.0},
 3665176.0: {Timestamp('2015-12-01 00:00:00'): 1.5,
  Timestamp('2018-10-01 00:00:00'): 1.5,
  Timestamp('2020-02-20 00:00:00'): 1.5},
 5111532.0: {Timestamp('2015-08-26 00:00:00'): 3.0,
  Timestamp('2015-03-25 00:00:00'): 3.0,
  Timestamp('2017-01-24 00:00:00'): 3.0},
 2943509.0: {Timestamp('2012-10-17 00:00:00'): 0.0,
  Timestamp('2013-01-23 00:00:00'): 0.0,
  Timestamp('2013-09-25 00:00:00'): 0.0,
  Timestamp('2015-03-25 00:00:00'): 0.0,
  Timestamp('2016-10-04 00:00:00'): 0.0,
  Timestamp('2019-02-05 00:00:00'): 0.0},
 3333502.0: {Timestamp('2016-07-13 00:00:00'): 2.0,
  Timestamp('2017-05-10 00:00:00'): 2.0},
 3115795.0: {Timestamp('2015-01-20 00:00:00'): 4.0,
  Timestamp('20

In [9]:
a = Counter(df_AI_labs['IR_QO'])
# print(a.most_common(80))
labs_selected = [x[0] for x in a.most_common()[:80]]


a = Counter(df_AI_vitals['IF_QO'])
# print(a.most_common(20))
vitals_selected = [x[0] for x in a.most_common()[:12]]

a = Counter(df_AI_medications['M_MedDict'])
# print(a.most_common(30))
meds_selected = [x[0] for x in a.most_common()[:12]]

In [76]:
### lab preprocess
df_AI_labs['R_ResultDTTM'] = pd.to_datetime(df_AI_labs['R_ResultDTTM'])
df_AI_labs = df_AI_labs[df_AI_labs['IR_QO'].isin(labs_selected)]
lab_names_mapping = {name: i for i, name in enumerate(labs_selected)}
### vital preprocess
PatientID_TO_MRN = {PatientID: MRN for PatientID, MRN in zip(df_AI_labs['PatientID'].to_list(), 
                                                             df_AI_labs['MRN'].to_list())}
df_AI_vitals = df_AI_vitals.replace({"PatientID": PatientID_TO_MRN}).rename(columns={'PatientID': 'MRN'})
df_AI_vitals['F_ClinicalDTTM'] = pd.to_datetime(df_AI_vitals['F_ClinicalDTTM'])
df_AI_vitals = df_AI_vitals[df_AI_vitals['IF_QO'].isin(vitals_selected)]
vital_names_mapping = {name: i for i, name in enumerate(vitals_selected)}
### med preprocess
PatientID_TO_MRN = {PatientID: MRN for PatientID, MRN in zip(df_AI_labs['PatientID'].to_list(), 
                                                             df_AI_labs['MRN'].to_list())}
df_AI_medications = df_AI_medications.replace({"PatientID": PatientID_TO_MRN}).rename(columns={'PatientID': 'MRN'})
df_AI_medications['IM_PerformedDTTM'] = pd.to_datetime(df_AI_medications['IM_PerformedDTTM'])
df_AI_medications = df_AI_medications[df_AI_medications['M_MedDict'].isin(meds_selected)]
med_names_mapping = {name: i for i, name in enumerate(meds_selected)}
### df_AI_demographics 
df_AI_demographics_2 = df_AI_demographics[['MRN', 'Race','Sex']]
df_AI_demographics_2 = pd.get_dummies(df_AI_demographics_2 , columns=['Race','Sex'])
### df_AI_patient_facts 
df_AI_patient_facts_2  = df_AI_patient_facts[['MRN', 'Race', 'Ethnicity']]
df_AI_patient_facts_2  = pd.get_dummies(df_AI_patient_facts_2, columns=['Race', 'Ethnicity'])

# MRI -> MRN
master_sheet = pd.read_excel('../ai_training_set/Updated Database.xlsx', sheet_name = '(6) MRI')
dict_MRI_MRN = {str(int(a)):str(int(b)) for a,b in  zip(master_sheet['ut_scan_id'].to_list(), 
                                                        master_sheet['allscripts_mrn'].to_list()) if not pd.isna(a) and not pd.isna(b)}

# MRI_Train is the MRI Image train set of MRIs
import pickle
with open('embeddings_T2/MRIs_train.pkl', 'rb') as f:
    train_MRIs = pickle.load(f)
with open('embeddings_T2/MRIs_test.pkl', 'rb') as f:
    test_MRIs = pickle.load(f)
T2_embedding_list_test = np.load('embeddings_T2/xgboost_embedding_list_test.npy')
T2_embedding_list_train = np.load('embeddings_T2/xgboost_embedding_list_train.npy')
T2_label_list_test = np.load('embeddings_T2/xgboost_label_list_test.npy')
T2_label_list_train = np.load('embeddings_T2/xgboost_label_list_train.npy')

T1_pre_embedding_list_test = np.load('embeddings_T1_pre/xgboost_embedding_list_test.npy')
T1_pre_embedding_list_train = np.load('embeddings_T1_pre/xgboost_embedding_list_train.npy')
T1_pre_label_list_test = np.load('embeddings_T1_pre/xgboost_label_list_test.npy')
T1_pre_label_list_train = np.load('embeddings_T1_pre/xgboost_label_list_train.npy')

T1_post_embedding_list_test = np.load('embeddings_T1_post/xgboost_embedding_list_test.npy')
T1_post_embedding_list_train = np.load('embeddings_T1_post/xgboost_embedding_list_train.npy')
T1_post_label_list_test = np.load('embeddings_T1_post/xgboost_label_list_test.npy')
T1_post_label_list_train = np.load('embeddings_T1_post/xgboost_label_list_train.npy')

flair_embedding_list_test = np.load('embeddings_flair/xgboost_embedding_list_test.npy')
flair_embedding_list_train = np.load('embeddings_flair/xgboost_embedding_list_train.npy')
flair_label_list_test = np.load('embeddings_flair/xgboost_label_list_test.npy')
flair_label_list_train = np.load('embeddings_flair/xgboost_label_list_train.npy')

pd_embedding_list_test = np.load('embeddings_pd/xgboost_embedding_list_test.npy')
pd_embedding_list_train = np.load('embeddings_pd/xgboost_embedding_list_train.npy')
pd_label_list_test = np.load('embeddings_pd/xgboost_label_list_test.npy')
pd_label_list_train = np.load('embeddings_pd/xgboost_label_list_train.npy')


assert np.array_equal(T2_label_list_train, T1_pre_label_list_train)
assert np.array_equal(T2_label_list_test , T1_pre_label_list_test )
assert np.array_equal( T2_label_list_train , T1_post_label_list_train) 
assert np.array_equal( T2_label_list_test , T1_post_label_list_test)
assert np.array_equal( T2_label_list_train , flair_label_list_train )
assert np.array_equal(  T2_label_list_test , flair_label_list_test)
assert np.array_equal( T2_label_list_train, pd_label_list_train )
assert np.array_equal(  T2_label_list_test , pd_label_list_test)


# Make MRN_train, which corresponds to MRI_Train
train_MRNs = [dict_MRI_MRN[i] for i in train_MRIs]
test_MRNs = [dict_MRI_MRN[i] for i in test_MRIs]

# make it a dictionary
d_mrn_embedding_train_T1_pre = {a:b for a,b in zip(train_MRNs, T1_pre_embedding_list_train)}
d_mrn_embedding_test_T1_pre = {a:b for a,b in zip(test_MRNs, T1_pre_embedding_list_test)}
d_mrn_embedding_train_T1_post = {a:b for a,b in zip(train_MRNs, T1_post_embedding_list_train)}
d_mrn_embedding_test_T1_post = {a:b for a,b in zip(test_MRNs, T1_post_embedding_list_test)}
d_mrn_embedding_train_T2 = {a:b for a,b in zip(train_MRNs, T2_embedding_list_train)}
d_mrn_embedding_test_T2 = {a:b for a,b in zip(test_MRNs, T2_embedding_list_test)}
d_mrn_embedding_train_flair= {a:b for a,b in zip(train_MRNs, flair_embedding_list_train)}
d_mrn_embedding_test_flair = {a:b for a,b in zip(test_MRNs, flair_embedding_list_test)}
d_mrn_embedding_train_pd = {a:b for a,b in zip(train_MRNs, pd_embedding_list_train)}
d_mrn_embedding_test_pd = {a:b for a,b in zip(test_MRNs, pd_embedding_list_test)}

## note embeddings
with open("../../Github_DocumentClassification-MPAD(my-code-Here)/mpad-master/mpad/result_MPAD_embedding.pickle", 'rb') as f:
    note_embeddings = pickle.load(f)
note_embeddings = {a:b for a,b in note_embeddings}


MS_train = []
MS_test = []

for mrn, time_value in tqdm(result_EDSS.items()):
    label = 0
    for time, value in time_value.items():
        time_reach_4 = time
        if value > 4.0: 
            label = 1
            break
        
    ###  ==================== lab ===========================
    n_col = len(lab_names_mapping)
    lab_i = [[0] * n_col]

    patient_lab = df_AI_labs[(df_AI_labs['MRN'] == mrn) & 
                             ((df_AI_labs['R_ResultDTTM'] < time_reach_4))].sort_values(by='R_ResultDTTM').reset_index(drop=True)

    if patient_lab.shape[0] != 0:
        prev_date = patient_lab.loc[0, 'R_ResultDTTM']

        for j in range(patient_lab.shape[0]):
            col = lab_names_mapping[patient_lab.loc[j, 'IR_QO']]
            val = patient_lab.loc[j, 'IR_DecodedValue']
            cur_date = patient_lab.loc[j, 'R_ResultDTTM']
            try: 
                val = float(val)
            except:
                continue
            if np.isnan(val):
                continue

            if cur_date != prev_date:
                lab_i.append([0] * n_col)
                prev_date = cur_date
            lab_i[-1][col] =val


    ###  ==================== vital===========================
    n_col = len(vital_names_mapping)
    vital_i = [[0] * n_col]

    patient_vital = df_AI_vitals[(df_AI_vitals['MRN'] == mrn) & ((df_AI_vitals['F_ClinicalDTTM'] < time_reach_4))].sort_values(by='F_ClinicalDTTM').reset_index(drop=True)

    if patient_vital.shape[0] != 0:
        prev_date = patient_vital.loc[0, 'F_ClinicalDTTM']

        for j in range(patient_vital.shape[0]):
            col = vital_names_mapping[patient_vital.loc[j, 'IF_QO']]
            val = patient_vital.loc[j, 'F_NormalizedValue']
            cur_date = patient_vital.loc[j, 'F_ClinicalDTTM']
            try: 
                val = float(val)
            except:
                continue
            if np.isnan(val):
                continue

            if cur_date != prev_date:
                vital_i.append([0] * n_col)
                prev_date = cur_date
            vital_i[-1][col] =val

    
    ###  ==================== medications ==========================
    n_col = len(med_names_mapping)
    med_i = [[0] * n_col]

    patient_med = df_AI_medications[(df_AI_medications['MRN'] == mrn) & ((df_AI_medications['IM_PerformedDTTM'] < time_reach_4))].sort_values(by='IM_PerformedDTTM').reset_index(drop=True)

    if patient_med.shape[0] != 0:
        prev_date = patient_med.loc[0, 'IM_PerformedDTTM']

        for j in range(patient_med.shape[0]):
            col = med_names_mapping[patient_med.loc[j, 'M_MedDict']]
            val = 1
            cur_date = patient_med.loc[j, 'IM_PerformedDTTM']
            try: 
                val = float(val)
            except:
                continue
            if np.isnan(val):
                continue

            if cur_date != prev_date:
                med_i.append([0] * n_col)
                prev_date = cur_date
            med_i[-1][col] =val

    
    ###  ==================== demographics ==========================
    demo_i_1 = df_AI_demographics_2[df_AI_demographics['MRN']==mrn].drop(columns='MRN').values.tolist()
    if not demo_i_1:
        demo_i_1 = [0] * len(list(df_AI_demographics_2.columns.drop('MRN')))
    else:
        demo_i_1 = demo_i_1[0]
    demo_i_2 = df_AI_patient_facts_2[df_AI_patient_facts['MRN']==mrn].drop(columns='MRN').values.tolist()
    if not demo_i_2:
        demo_i_2 = [0] * len(list(df_AI_patient_facts_2.columns.drop('MRN')))
    else:
        demo_i_2 = demo_i_2[0]
    demo_i = demo_i_1 +  demo_i_2
    
    
    ### ========================== Note Embeddings ====================================
    note_i = note_embeddings[mrn]

    ### ======================= EDSS score ===========================
    label_i = label


    ### ==================== MRI Embeddings ==========================
    ### ===== if MRN has image data and in train =====
    if str(int(mrn)) in d_mrn_embedding_train:
        mri_embed_T1_pre_i = d_mrn_embedding_train_T1_pre[str(int(mrn))]
        mri_embed_T1_post_i = d_mrn_embedding_train_T1_post[str(int(mrn))]
        mri_embed_T2_i = d_mrn_embedding_train_T2[str(int(mrn))]
        mri_embed_flair_i = d_mrn_embedding_train_flair[str(int(mrn))]
        mri_embed_pd_i = d_mrn_embedding_train_pd[str(int(mrn))]
        MS_train.append((str(int(mrn)), (torch.tensor(lab_i, dtype = torch.float32), 
                                         torch.tensor(vital_i, dtype = torch.float32), 
                                         torch.tensor(med_i, dtype = torch.float32), 
                                         torch.tensor(note_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_T1_pre_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_T1_post_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_T2_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_flair_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_pd_i, dtype = torch.float32), 
                                         torch.tensor(demo_i, dtype = torch.float32), 
                                         label_i)))
    ### ====== if MRN has image data and in test =======
    elif str(int(mrn)) in d_mrn_embedding_test:
        mri_embed_T1_pre_i = d_mrn_embedding_test_T1_pre[str(int(mrn))]
        mri_embed_T1_post_i = d_mrn_embedding_test_T1_post[str(int(mrn))]
        mri_embed_T2_i = d_mrn_embedding_test_T2[str(int(mrn))]
        mri_embed_flair_i = d_mrn_embedding_test_flair[str(int(mrn))]
        mri_embed_pd_i = d_mrn_embedding_test_pd[str(int(mrn))]
        MS_test.append((str(int(mrn)), (torch.tensor(lab_i, dtype = torch.float32), 
                                         torch.tensor(vital_i, dtype = torch.float32), 
                                         torch.tensor(med_i, dtype = torch.float32), 
                                         torch.tensor(note_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_T1_pre_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_T1_post_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_T2_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_flair_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_pd_i, dtype = torch.float32), 
                                         torch.tensor(demo_i, dtype = torch.float32), 
                                         label_i)))
    ### ========= if MRN don't have image, create empty image embedding and put this patient randomly to train/test
    else:
        mri_embed_T1_pre_i = np.zeros(shape=(258,))
        mri_embed_T1_post_i = np.zeros(shape=(258,))
        mri_embed_T2_i = np.zeros(shape=(258,))
        mri_embed_flair_i = np.zeros(shape=(258,))
        mri_embed_pd_i = np.zeros(shape=(258,))
        if np.random.random() < 0.5:
            MS_train.append((str(int(mrn)), (torch.tensor(lab_i, dtype = torch.float32), 
                                         torch.tensor(vital_i, dtype = torch.float32), 
                                         torch.tensor(med_i, dtype = torch.float32), 
                                         torch.tensor(note_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_T1_pre_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_T1_post_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_T2_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_flair_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_pd_i, dtype = torch.float32), 
                                         torch.tensor(demo_i, dtype = torch.float32), 
                                         label_i)))
        else:
            MS_test.append((str(int(mrn)), (torch.tensor(lab_i, dtype = torch.float32), 
                                         torch.tensor(vital_i, dtype = torch.float32), 
                                         torch.tensor(med_i, dtype = torch.float32), 
                                         torch.tensor(note_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_T1_pre_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_T1_post_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_T2_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_flair_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_pd_i, dtype = torch.float32), 
                                         torch.tensor(demo_i, dtype = torch.float32), 
                                         label_i)))
            

    
    

FileNotFoundError: [Errno 2] No such file or directory: 'embeddings_T2/MRIs_train.pkl'

In [ ]:
df_AI_demographics_2[df_AI_demographics['MRN']==mrn].drop(columns='MRN')

#### MRI——》MRN是多对1的关系

In [ ]:
len(set(train_MRIs)),\
len(set(test_MRIs)),\
len(set(train_MRIs).intersection(set(test_MRIs)))

In [77]:
len(set(train_MRNs)),\
len(set(test_MRNs)),\
len(set(train_MRNs).intersection(set(test_MRNs)))

(133, 136, 50)

In [78]:
p = 0
n = 0
na = 0
for mrn, time_value in tqdm(result_EDSS.items()):
    if str(int(mrn)) in d_mrn_embedding_train:
#          print('+')
         p+= 1
    elif str(int(mrn)) in d_mrn_embedding_test:
#          print('-')
         n+=1
    else:
#         print('no')
        na+=1
print(p, n, na)

100%|██████████| 299/299 [00:00<00:00, 583843.99it/s]

133 86 80


In [79]:
len(d_mrn_embedding_train.keys()), \
len(d_mrn_embedding_test.keys()), \
len(set(d_mrn_embedding_train.keys()).intersection(set(d_mrn_embedding_test.keys())))

(133, 136, 50)

In [80]:
d_mrn_embedding_train.keys()

dict_keys(['3227903', '922265', '5493039', '2893162', '107204', '3377497', '2926524', '2668311', '3866533', '2739843', '376911', '2945031', '2943509', '1560280', '3735728', '4276751', '1800779', '1876883', '1568530', '5609457', '3833276', '372306', '3661950', '2975422', '5823210', '2731475', '3784371', '3673138', '1296434', '6316834', '1950367', '4659102', '774561', '2484244', '5824707', '1243867', '3900749', '362197', '4904372', '6048204', '4665129', '3825330', '689271', '2296815', '5380728', '2682180', '2188643', '4247516', '3560955', '67552', '2857932', '283833', '3709272', '5370173', '3531437', '3390284', '5014511', '4253586', '5056858', '3334154', '2429412', '5951066', '2298557', '2790928', '1159182', '2930796', '3910324', '352331', '2741782', '3024396', '4195424', '5377245', '5912851', '318608', '918070', '427966', '3921116', '680805', '3190885', '12532', '3970508', '3450842', '3364218', '1388589', '5271797', '4563136', '5987756', '3561084', '3785501', '5134990', '6429427', '6324

In [81]:
d_mrn_embedding_test.keys()

dict_keys(['5973826', '5823210', '3736364', '5824707', '3712550', '4863758', '3518845', '2218454', '3983587', '4782217', '3450581', '4994919', '5957968', '1159182', '3742444', '2022253', '6124056', '3933314', '3561084', '1388589', '1726826', '3709272', '3735728', '3364218', '6161748', '6048204', '4546332', '3749047', '3900993', '6033740', '991178', '4186417', '1800779', '633190', '1610725', '4676116', '3901960', '4000055', '2665966', '3255249', '3670337', '4645913', '1883537', '6345731', '5196810', '5772458', '4306821', '3440051', '3828778', '4659102', '4268337', '4033058', '3869548', '108532', '5185285', '5111532', '3790917', '2429412', '3190885', '2253682', '4518775', '3781214', '1237033', '2566485', '3255126', '2731475', '3069102', '3884653', '365376', '4106333', '416918', '4218610', '5517417', '3177428', '2564323', '2343584', '2945031', '3910324', '3833276', '2790928', '5014511', '2519029', '4665129', '3783619', '3377497', '3115795', '3534077', '1798431', '3549686', '2229898', '129

In [82]:
len(MS_train), len(MS_test)

(167, 132)

In [83]:
with open('MS_train.pt', 'wb') as f:
    pickle.dump(MS_train, f)
with open('MS_test.pt', 'wb') as f:
    pickle.dump(MS_test, f)

In [84]:
import pickle
with open('MS_train.pt', 'rb') as f:
    MS_train = pickle.load(f)
with open('MS_test.pt', 'rb') as f:
    MS_test = pickle.load(f)

In [85]:
for i in range(10):
    print(MS_train[0][1][i].shape)

torch.Size([3, 80])
torch.Size([14, 12])
torch.Size([1, 12])
torch.Size([256])
torch.Size([258])
torch.Size([258])
torch.Size([258])
torch.Size([258])
torch.Size([258])
torch.Size([21])


In [86]:
print(len(result_EDSS))
print(len(d_mrn_embedding_train), len(d_mrn_embedding_test))

299
133 136


In [87]:
d_mrn_embedding_train.keys(), d_mrn_embedding_test.keys()

(dict_keys(['3227903', '922265', '5493039', '2893162', '107204', '3377497', '2926524', '2668311', '3866533', '2739843', '376911', '2945031', '2943509', '1560280', '3735728', '4276751', '1800779', '1876883', '1568530', '5609457', '3833276', '372306', '3661950', '2975422', '5823210', '2731475', '3784371', '3673138', '1296434', '6316834', '1950367', '4659102', '774561', '2484244', '5824707', '1243867', '3900749', '362197', '4904372', '6048204', '4665129', '3825330', '689271', '2296815', '5380728', '2682180', '2188643', '4247516', '3560955', '67552', '2857932', '283833', '3709272', '5370173', '3531437', '3390284', '5014511', '4253586', '5056858', '3334154', '2429412', '5951066', '2298557', '2790928', '1159182', '2930796', '3910324', '352331', '2741782', '3024396', '4195424', '5377245', '5912851', '318608', '918070', '427966', '3921116', '680805', '3190885', '12532', '3970508', '3450842', '3364218', '1388589', '5271797', '4563136', '5987756', '3561084', '3785501', '5134990', '6429427', '632

In [88]:
print(len(train_MRIs)), print(len(test_MRIs))
print(len(train_MRNs)), print(len(test_MRNs))


353
173
353
173


(None, None)

In [101]:
labs_selected[39]

'ABSOLUTE NEUTROPHILS'

In [108]:
tmp = []
for i in labs_selected:
#     print("'"+i.split(' ')[0]+"',")
    tmp.append(i)
print(tmp)

tmp = []
for i in vitals_selected:
#     print("'"+i.split(' ')[0]+"',")
    tmp.append(i)
print(tmp)

tmp = []
for i in meds_selected:
#     print("'"+i.split(' ')[0]+"',")
    tmp.append(i)
print(tmp)


['MCH', 'RDW', 'MCHC', 'MCV', 'ALT', 'AST', 'WBC', 'RBC', 'ABSOLUTE LYMPHOCYTES', 'Monocytes', 'Hct', 'Hgb', 'Platelet', 'Mean Platelet Volume', 'A/G Ratio', 'Lymphocytes #', 'Monocytes #', 'Segs-Bands #', 'Lymphocytes', 'Segmented Neutrophils', 'Eosinophils', 'Basophils', 'WHITE BLOOD CELL COUNT', 'HEMATOCRIT', 'RED BLOOD CELL COUNT', 'PLATELET COUNT', 'Total Protein', 'Bili Total', 'Alk Phos', 'Albumin Lvl', 'Globulin', 'NEUTROPHILS', 'LYMPHOCYTES', 'ABSOLUTE EOSINOPHILS', 'BASOPHILS', 'ABSOLUTE MONOCYTES', 'EOSINOPHILS', 'ABSOLUTE BASOPHILS', 'MONOCYTES', 'ABSOLUTE NEUTROPHILS', 'ALBUMIN', 'PROTEIN, TOTAL', 'BILIRUBIN, TOTAL', 'ALBUMIN/GLOBULIN RATIO', 'GLOBULIN', 'Eosinophils #', 'MPV', 'MRI BRAIN W WO CONTRAST', 'GLUCOSE', 'BUN/Creatinine Ratio', 'HEMAGLOBIN', 'Creatinine Lvl', 'Blood Urea Nitrogen', 'Glucose Lvl', 'Calcium Level Total', 'Sodium Level', 'Potassium Level', 'Carbon Dioxide', 'Chloride Level', 'AGAP', 'eGFR', 'CREATININE', 'ALKALINE PHSPHATASE', 'SODIUM', 'CALCIUM', 